In [1]:
import numpy as np
import math

from autograd import grad, jacobian
import scipy.linalg as sp_linalg
import autograd.numpy as np

### Define physical constants

In [2]:
# Motor
Rm = 8.4  # Resistance
kt = 0.042  # Current-torque (N-m/A)
km = 0.042  # Back-emf constant (V-s/rad)

# Rotary Arm
mr = 0.095  # Mass (kg)
Lr = 0.085  # Total length (m)
Jr = mr * Lr ** 2 / 12  # Moment of inertia about pivot (kg-m^2)
Dr = 0.0015  # Equivalent viscous damping coefficient (N-m-s/rad)

# Pendulum Link
mp = 0.024  # Mass (kg)
Lp = 0.129  # Total length (m)
Jp = mp * Lp ** 2 / 12  # Moment of inertia about pivot (kg-m^2)
Dp = 0.0005  # Equivalent viscous damping coefficient (N-m-s/rad)

g = 9.81  # Gravity constant

### Create the simulator

In [3]:
def forward_model(state, action, dt=1/250.0):
    Vm = action
    theta, alpha, theta_dot, alpha_dot = state
    tau = (km * (Vm - km * theta_dot)) / Rm  # torque of rotary arm

    # Copied from derive notebook
    theta_dot_dot = (-Lp*Lr*mp*(-8.0*Dp*alpha_dot + Lp**2*mp*theta_dot**2*np.sin(2.0*alpha) + 4.0*Lp*g*mp*np.sin(alpha))*np.cos(alpha) + (4.0*Jp + Lp**2*mp)*(4.0*Dr*theta_dot + Lp**2*mp*alpha_dot*theta_dot*np.sin(2.0*alpha) + 2.0*Lp*Lr*mp*alpha_dot**2*np.sin(alpha) - 4.0*tau))/(4.0*Lp**2*Lr**2*mp**2*np.cos(alpha)**2 - (4.0*Jp + Lp**2*mp)*(4.0*Jr + Lp**2*mp*np.sin(alpha)**2 + 4.0*Lr**2*mp))
    alpha_dot_dot = (2.0*Lp*Lr*mp*(4.0*Dr*theta_dot + Lp**2*mp*alpha_dot*theta_dot*np.sin(2.0*alpha) + 2.0*Lp*Lr*mp*alpha_dot**2*np.sin(alpha) - 4.0*tau)*np.cos(alpha) - 0.5*(4.0*Jr + Lp**2*mp*np.sin(alpha)**2 + 4.0*Lr**2*mp)*(-8.0*Dp*alpha_dot + Lp**2*mp*theta_dot**2*np.sin(2.0*alpha) + 4.0*Lp*g*mp*np.sin(alpha)))/(4.0*Lp**2*Lr**2*mp**2*np.cos(alpha)**2 - (4.0*Jp + Lp**2*mp)*(4.0*Jr + Lp**2*mp*np.sin(alpha)**2 + 4.0*Lr**2*mp))

    # Semi-implicit euler
    theta_dot += theta_dot_dot * dt
    alpha_dot += alpha_dot_dot * dt
    theta += theta_dot * dt
    alpha += alpha_dot * dt

    # Normalize in range [0, 2π]
    theta %= 2 * np.pi
    alpha %= 2 * np.pi

    # For discrete version of LQR
    state = np.array([theta, alpha, theta_dot, alpha_dot])
    return state

# Test forward_model
init_state = [0, np.pi, 0, 0]
forward_model(init_state, 0.5, dt=0.05)

array([ 0.06215942,  3.08015602,  1.24318836, -1.22873269])

### Linearize the dynamics around point "current_state"

In [4]:
def computeAB(current_state, current_control):
    forward_dynamics_model = lambda state, action: forward_model(state, action)
    a_mat = jacobian(forward_dynamics_model, 0)
    b_mat = jacobian(forward_dynamics_model, 1)
    A = a_mat(current_state, current_control)
    B = b_mat(current_state, current_control)

    # Correct continuous time linearization from Quanser Workbook
    # A = [[0, 0, 1.0000, 0], [0,0,0 ,1.0000], [0,149.2751,-0.0104,0], [0,261.6091,-0.0103,0]]
    # B = [0, 0, 49.7275, 49.1493]
    return A, B

### LQR using a ricatti equation

In [5]:
def LQR_control():
    # Cost matrices for LQR
    Q = np.diag(np.array([1, 1, 1, 1]))  # state_dimension = 4
    R = np.eye(1)  # control_dimension = 1

    # Linearize the around point [0,0,0,0] (upright stationary), with action [0]
    A, B = computeAB(np.array([0.0, 0.0, 0.0, 0.0]), np.array([0.0]))

    # Use if discrete forward dynamics is used
    X = sp_linalg.solve_discrete_are(A, B, Q, R)
    K = np.dot(np.linalg.pinv(R + np.dot(B.T, np.dot(X, B))), np.dot(B.T, np.dot(X, A)))

    return np.squeeze(K, 0)

### Compare Quanser's gains to derived gains
(the two should be similar)

In [6]:
K_real = [-2.0, 35.0, -1.5, 3.0]  # Correct K from quanser workbook
K_calc = LQR_control().tolist()
print("The two following should be close to each other")
print("\tThe gains from Quanser are:", K_real)
print("\tThe calculated gains are:  ", K_calc)

TypeError: return arrays must be of ArrayType